# Explore here

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib as os


from sklearn.metrics.pairwise import cosine_similarity

## Carga del Dataset

In [3]:
url = "https://raw.githubusercontent.com/4GeeksAcademy/predicting-your-future-with-data/main/adult-census-income.csv"
df = pd.read_csv(url, delimiter=",")  
df.to_csv("/workspaces/Jacinto-Sistemas-de-Recomendaci-n/src/adult-census-income.csv", index=False)

## 1 - Revisión Inicial

In [4]:
# 1.1 - Revisamos las primeras filas
df.head() 

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [5]:
# 1.2 - Comprobar dimensiones y tipos de datos

# Número de filas y columnas
print(f"El dataset tiene {df.shape[0]} filas y {df.shape[1]} columnas.")

# Información general del DataFrame
df.info()

El dataset tiene 32561 filas y 15 columnas.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education.num   32561 non-null  int64 
 5   marital.status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital.gain    32561 non-null  int64 
 11  capital.loss    32561 non-null  int64 
 12  hours.per.week  32561 non-null  int64 
 13  native.country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [6]:
# 1.3 - Observamos los valores unicos de cada columna del df
n_unicos = df.nunique()
n_unicos.sort_values(ascending=False)
n_unicos

age                  73
workclass             9
fnlwgt            21648
education            16
education.num        16
marital.status        7
occupation           15
relationship          6
race                  5
sex                   2
capital.gain        119
capital.loss         92
hours.per.week       94
native.country       42
income                2
dtype: int64

In [7]:
# 1.4 - Comprobar la presencia de filas duplicadas
duplicados = df.duplicated().sum()
print(f"Filas duplicadas encontradas: {duplicados}")

if duplicados > 0:
    df = df.drop_duplicates()
    print(f"Se eliminaron {duplicados} duplicados. Tamaño final: {df.shape}")


Filas duplicadas encontradas: 24
Se eliminaron 24 duplicados. Tamaño final: (32537, 15)


## 2 - Preprocesamiento de Datos

In [8]:
# 2.1 — Reemplazar valores mal codificados con np.nan
df.replace("?", np.nan, inplace=True)

# 2.2 — Comprobación de nulos
print("Valores nulos por columna:")
print(df.isnull().sum())

# 2.3 — Eliminamos filas con valores nulos
df = df.dropna().copy()
print(f"\nTamaño tras eliminar nulos: {df.shape}")

# 2.4 — Codificamos variables categóricas con One-Hot Encoding
cat_cols = df.select_dtypes(include="object").columns.drop("income")
df_encoded = pd.get_dummies(df, columns=cat_cols)

# 2.5 — Convertimos variable objetivo a binaria
df_encoded["income"] = df_encoded["income"].map({">50K": 1, "<=50K": 0})

# Visualización final de estructura
print(f"\nDataFrame preprocesado con shape: {df_encoded.shape}")
df_encoded.head()

Valores nulos por columna:
age                  0
workclass         1836
fnlwgt               0
education            0
education.num        0
marital.status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital.gain         0
capital.loss         0
hours.per.week       0
native.country     582
income               0
dtype: int64

Tamaño tras eliminar nulos: (30139, 15)

DataFrame preprocesado con shape: (30139, 105)


,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week,income,workclass_Federal-gov,workclass_Local-gov,workclass_Private,...,native.country_Portugal,native.country_Puerto-Rico,native.country_Scotland,native.country_South,native.country_Taiwan,native.country_Thailand,native.country_Trinadad&Tobago,native.country_United-States,native.country_Vietnam,native.country_Yugoslavia
1,82,132870,9,0,4356,18,0,False,False,True,...,False,False,False,False,False,False,False,True,False,False
3,54,140359,4,0,3900,40,0,False,False,True,...,False,False,False,False,False,False,False,True,False,False
4,41,264663,10,0,3900,40,0,False,False,True,...,False,False,False,False,False,False,False,True,False,False
5,34,216864,9,0,3770,45,0,False,False,True,...,False,False,False,False,False,False,False,True,False,False
6,38,150601,6,0,3770,40,0,False,False,True,...,False,False,False,False,False,False,False,True,False,False


## 3 - Definición del sistema de recomendación

El objetivo es recomendar trayectorias de mejora laboral y educativa que aumenten las probabilidades de tener ingresos anuales superiores a $50K.

Se pueden interpretar las recomendaciones como sugerencias de:
- Ocupaciones potenciales
- Niveles educativos alcanzables
- Cambios en horas trabajadas semanales


Cada “usuario” es una persona con un conjunto de características socioeconómicas: edad, nivel de educación actual, estado civil, ocupación actual, sexo, país de origen, etc.

Estas variables se usarán para construir un vector de características que represente su perfil.

Para generar recomendaciones, se utilizarán como base las variables:

- age
- education / education.num
- marital.status
- occupation
- hours.per.week
- sex
- native.country

Este perfil servirá como punto de entrada para buscar otras personas similares en el dataset y ver qué combinaciones de variables están asociadas con mayores ingresos.

Con esto definido construiremos el sistema de recomendación propiamente dicho, basándonos en similitud de perfiles.

## 4 - Construcción del sistema de recomendación

In [ ]:
# Seleccionamos X ya sin la variable 'income'
X = df_encoded.drop("income", axis=1)

# Simulamos un perfil (ejemplo rápido)
usuario_simulado = X.sample(1, random_state=42)  # 1 usuario aleatorio

# Calculamos similitud del usuario con el resto
similitudes = cosine_similarity(usuario_simulado, X)

# Obtenemos los 10 perfiles más parecidos
top_indices = similitudes[0].argsort()[::-1][1:11]
usuarios_similares = df.iloc[top_indices]

usuarios_similares[["occupation", "education", "hours.per.week", "income"]]

,occupation,education,hours.per.week,income
5185,Exec-managerial,Bachelors,50,<=50K
25675,Tech-support,Bachelors,45,>50K
5175,Sales,Bachelors,50,>50K
6869,Machine-op-inspct,HS-grad,40,<=50K
22905,Sales,Masters,65,>50K
22417,Exec-managerial,HS-grad,40,<=50K
15138,Exec-managerial,Bachelors,45,>50K
17786,Sales,Bachelors,60,>50K
19961,Prof-specialty,Doctorate,60,>50K
23647,Prof-specialty,Doctorate,60,>50K


### Resultado de recomendación para perfil simulado

Tras simular un perfil y calcular su similitud con el resto de usuarios del dataset, se identificaron los 10 perfiles más parecidos. Al observar las trayectorias de esos individuos, se detectan patrones claros:

- La mayoría tiene al menos educación universitaria (Bachelors, Masters o Doctorate)
- Las ocupaciones más frecuentes entre los que superan los $50K son:
  - Executive Managerial
  - Sales
  - Professional Specialty
  - Tech Support
- Las jornadas laborales también tienden a ser más intensas: entre 45 y 60 horas semanales

### Conclusión

El sistema de recomendación sugiere que, para perfiles similares, aumentar la formación académica y aspirar a determinadas ocupaciones técnicas o de gestión, puede estar relacionado con un mayor nivel de ingresos.

Este análisis sirve de base para construir recomendaciones más específicas según objetivos o perfil actual.

## 5 - Pruebas con casos simulados

Se crean perfiles artificiales de usuarios con distintas características socioeconómicas y laborales. Luego, utilizando el DataFrame codificado, se comparan con todos los usuarios reales del dataset mediante cosine_similarity.

El objetivo es detectar qué trayectorias han seguido individuos similares que obtienen ingresos superiores a $50K, para generar recomendaciones fundamentadas.

Este tipo de prueba permite validar si el sistema ofrece recomendaciones coherentes y personalizadas para cada tipo de perfil.

In [ ]:
# Crear perfil simulado

perfil_usuario = {
    "age": 25,
    "fnlwgt": 200000,  # peso muestral arbitrario
    "education.num": 9,  # equivalente a HS-grad
    "capital.gain": 0,
    "capital.loss": 0,
    "hours.per.week": 20,
    "sex_Male": 1,
    "sex_Female": 0,
    "workclass_Private": 1,
    "workclass_Self-emp-not-inc": 0,
    "marital.status_Never-married": 1,
    "marital.status_Married-civ-spouse": 0,
    # ... y así para el resto de columnas codificadas
}

# 1. Crear un DataFrame con las mismas columnas que df_encoded (excepto income)
columnas_modelo = df_encoded.drop("income", axis=1).columns
perfil_vector = pd.DataFrame([perfil_usuario], columns=columnas_modelo).fillna(0)

# 2. Calcular similitud con cada usuario del dataset
X = df_encoded.drop("income", axis=1)
similitudes = cosine_similarity(perfil_vector, X)

# 3. Elegir los N usuarios más similares
top_n = 10
indices_similares = similitudes[0].argsort()[::-1][:top_n]
usuarios_similares = df.iloc[indices_similares]

# 4. Mostrar posibles recomendaciones
usuarios_similares[["occupation", "education", "hours.per.week", "income"]]

,occupation,education,hours.per.week,income
24029,Prof-specialty,Doctorate,40,>50K
29045,Transport-moving,Assoc-acdm,30,<=50K
23916,Exec-managerial,Doctorate,40,>50K
17431,Farming-fishing,Some-college,20,<=50K
22318,Other-service,Some-college,20,<=50K
31987,Prof-specialty,Masters,40,>50K
12494,Prof-specialty,Bachelors,25,>50K
29414,Other-service,Some-college,30,<=50K
4760,Craft-repair,Masters,40,>50K
17815,Exec-managerial,Masters,40,>50K


### Recomendaciones para perfil simulado: joven con estudios secundarios y media jornada

A partir del análisis de los 10 usuarios reales más similares a nuestro perfil simulado (25 años, HS-grad, 20h/semana), se observan los siguientes patrones:

- Ocupaciones relacionadas con mayores ingresos (>50K):
  - Professional Specialty
  - Executive Managerial
  - Craft-repair

- Formación académica destacada:
  - La mayoría tienen estudios universitarios (Bachelors o más)

- Jornadas laborales más extensas:
  - Entre 40 y 60 horas por semana

### Recomendación basada en datos

El sistema sugiere que, para mejorar sus ingresos, el usuario podría considerar:
- Incrementar su nivel educativo (ej. acceder a una carrera técnica o universitaria)
- Aspirar a ocupaciones especializadas
- Aumentar su jornada laboral si es viable

Este análisis demuestra que el modelo no solo detecta patrones, sino que proporciona orientación realista y accionable basada en casos similares.

## 6 - Exportación del dataset preprocesado

In [ ]:
# Crear carpeta si no existe
os.makedirs("data", exist_ok=True)

# Ahora sí, guardar el archivo
df_encoded.to_csv("data/dataset_preparado.csv", index=False)
print("Archivo guardado correctamente en 'data/dataset_preparado.csv'")

Archivo guardado correctamente en 'data/dataset_preparado.csv'
